In [34]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
%matplotlib inline
import argparse
import os
import pprint
import shutil
import pickle
import cv2
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import random

import torch
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
#from tensorboardX import SummaryWriter

import _init_paths
from config import cfg
from config import update_config
from core.loss import JointsMSELoss
from core.function import train
from core.function import validate
from utils.utils import get_optimizer
from utils.utils import save_checkpoint
from utils.utils import create_logger
from utils.utils import get_model_summary

import dataset
import models

In [35]:
with open('/u/snaha/v6/dataset/AWA/Animals_with_Attributes2/quadruped_keypoints/coco_format/dataset.pickle', 'rb') as handle:
    dataset = pickle.load(handle)
    
dataset.keys()

dict_keys(['annotations', 'images', 'categories', 'info'])

In [36]:
dataset['images'][0]

{'width': 1024,
 'height': 768,
 'filename': 'antelope_10002.jpg',
 'id': 'antelope_10002'}

In [37]:
path = '/u/snaha/v6/dataset/AWA/Animals_with_Attributes2/quadruped_keypoints/Annotations/'
list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]

animal_list = []

for animal in list_subfolders_with_paths:
    animal_folder = animal#os.path.join(path, animal)
    animal_list.append(animal_folder.split('/')[-1])
    
animal_list    

['antelope',
 'bobcat',
 'buffalo',
 'chihuahua',
 'collie',
 'cow',
 'dalmatian',
 'deer',
 'elephant',
 'fox',
 'german+shepherd',
 'giant+panda',
 'giraffe',
 'grizzly+bear',
 'hippopotamus',
 'horse',
 'leopard',
 'lion',
 'moose',
 'otter',
 'ox',
 'persian+cat',
 'pig',
 'polar+bear',
 'rabbit',
 'raccoon',
 'rhinoceros',
 'sheep',
 'siamese+cat',
 'skunk',
 'squirrel',
 'tiger',
 'weasel',
 'wolf',
 'zebra']

In [38]:
def generate_split(animal_list, dataset):
    
    rand_inds = np.arange(len(animal_list)).tolist()
    random.shuffle(rand_inds)    
    anim_list = []
    for ind in rand_inds:
        anim_list.append(animal_list[ind])
    dataset_train = dict()
    dataset_test = dict() 
    
    annotations_train = []
    annotations_test = []
    
    images_train = []
    images_test = []
    
    train_animals = anim_list[:-4]
    test_animal = anim_list[-4:]
    
    for anno in dataset['annotations']:
        if anno['animal'] in test_animal:
            annotations_test.append(anno)
        else:
            annotations_train.append(anno)
        
    for anno in dataset['images']:
        if anno['id'].split('_')[0] in test_animal:
            images_test.append(anno)
        else:
            images_train.append(anno)        
                
    dataset_train['annotations'] = annotations_train
    dataset_test['annotations'] = annotations_test
    
    dataset_train['images'] = images_train
    dataset_test['images'] = images_test
    
    dataset_train['categories'] = dataset['categories']
    dataset_test['categories'] = dataset['categories']    
    
    dataset_train['info'] = dataset['info']
    dataset_test['info'] = dataset['info']       
    
    return train_animals, test_animal, dataset_train, dataset_test

for i in range(5):
    train_animals, test_animals, dataset_train, dataset_test = generate_split(animal_list, dataset)
    #test_animals
    print('Number of train instances = ', len(dataset_train['images']), len(dataset_train['annotations']))
    print('Number of test instances = ', len(dataset_test['images']), len(dataset_test['annotations']))
    
    with open('/u/snaha/v6/dataset/AWA/Animals_with_Attributes2/quadruped_keypoints/coco_format/dataset_train_' + str(i+1) + '.pickle', 'wb') as handle:
        pickle.dump(dataset_train, handle, protocol=pickle.HIGHEST_PROTOCOL)    
        
    with open('/u/snaha/v6/dataset/AWA/Animals_with_Attributes2/quadruped_keypoints/coco_format/dataset_val_' + str(i+1) + '.pickle', 'wb') as handle:
        pickle.dump(dataset_test, handle, protocol=pickle.HIGHEST_PROTOCOL)            

Number of train instances =  8877 8877
Number of test instances =  1153 1153
Number of train instances =  8833 8833
Number of test instances =  1197 1197
Number of train instances =  8813 8813
Number of test instances =  1217 1217
Number of train instances =  8838 8838
Number of test instances =  1192 1192
Number of train instances =  8852 8852
Number of test instances =  1178 1178


In [39]:
a = [.24, .72, .72, .18, .62, .23, .56, .26, .38, .28, .29, .48, .23, .60, .47, .24, .72, .72, .18, .62, .23, .56, .26, .38, .28, .29, .48, .23, .60, .47, .18, .62, .23, .56, .26, .38, .28, .29, .48]
len(a)

39